# Universities Data Analysis

## Importing Dependencies

In [1]:
import sqlalchemy
import pandas as pd
from pandas_dedupe import dedupe_dataframe

We will be using:

- `sqlalchemy` to create a connection to the database.
- `pandas` to load SQL data to dataframes and do data manipulation.
- [`pandas-dedupe`](https://pypi.org/project/pandas-dedupe/) to assist us with de-duplication (particularly the "subject" column).

## Creating a Database Connection

In [2]:
def create_engine(username, password, hostname, port, database):
    uri = f'postgresql://{username}:{password}@{hostname}:{port}/{database}'
    return sqlalchemy.create_engine(uri)

engine = create_engine('postgres', 'postgres', 'localhost', 5432, 'universities')

## Loading the Data from the Database

In [3]:
raw_grades_df = pd.read_sql_query('SELECT * FROM raw_grades', engine)
raw_grades_df.sample(n=12)

,first_name,last_name,subject,grade,university
7822,gprozlpunq,n,Biology,16.0,uni2
17098,v,ob,Soc. Studies,None,uni3
6398,sdgc,z,Biology,49.0,uni2
18307,axd,tsmfriv,Bus. Administration,45.0,uni3
4810,scv,ecbq,Music,None,uni1
4529,qaj,mdwzzjswx,History,C,uni1
16174,aktzvzmhne,ocpinx,Soc. Studies,49.0,uni3
18878,ptozk,uahzbppl,Mathematics,36.0,uni3
247,twru,blrnrg,Physics,C,uni1
1365,rn,naitbjikq,Art,F,uni1


In [4]:
raw_grades_df.shape

(20480, 5)

We have over 20K records in total.

## De-duplicating Records

In [5]:
subjects_df = pd.read_sql_query('SELECT DISTINCT(subject) FROM raw_grades', engine)
subjects_df

,subject
0,Economics
1,Art
2,Soc. Studies
3,History
4,Music
5,Computer Science
6,Soc Studies
7,Bus. Administration
8,Social Studies
9,Biology


These are all the distinct values from "subject" encountered in the data. Given the small volume of data, it's pretty straight-forward to do deduplication in this case, but we need to find a method that scales for larger amounts of data. For this reaason, we will be using `pandas-dedupe`:

In [6]:
subjects_df = dedupe_dataframe(subjects_df, ['subject'], canonicalize=True)

Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...


/home/franco/PycharmProjects/kheiron-medical/university-pipeline/.venv/lib/python3.8/site-packages/pandas_dedupe/utility_functions.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df[i] = df[i].str.replace('[^\w\s\.\-\(\)\,\:\/\\\\]','')


# duplicate sets 13


The warning `FutureWarning` is harmless (for now) and it's due to `pandas` deprecation. That's the result of applying de-duplication over a list of distinct subjects:

In [7]:
subjects_df

,subject,cluster id,confidence,canonical_subject
0,economics,1,1.000000,economics
1,art,2,1.000000,art
2,soc. studies,0,0.986414,soc studies
3,history,3,1.000000,history
4,music,4,1.000000,music
5,computer science,5,1.000000,computer science
6,soc studies,0,0.999482,soc studies
7,bus. administration,6,1.000000,bus. administration
8,social studies,0,0.986404,soc studies
9,biology,7,1.000000,biology


The package was unable to match "maths" and "mathematics", so we can set this manually:

In [8]:
subjects_df.loc[subjects_df['canonical_subject'] == 'maths', 'canonical_subject'] = 'mathematics'
subjects_df

,subject,cluster id,confidence,canonical_subject
0,economics,1,1.000000,economics
1,art,2,1.000000,art
2,soc. studies,0,0.986414,soc studies
3,history,3,1.000000,history
4,music,4,1.000000,music
5,computer science,5,1.000000,computer science
6,soc studies,0,0.999482,soc studies
7,bus. administration,6,1.000000,bus. administration
8,social studies,0,0.986404,soc studies
9,biology,7,1.000000,biology


We will save this dataframe to a table named "subjects" to de-duplicate the table "raw_grades":

In [9]:
subjects_df.to_sql('subjects', engine, if_exists= 'replace')

We will create a new table named "grades" which is de-duplicated:

In [10]:
engine.execute(
    """
    DROP TABLE IF EXISTS grades;
    CREATE TABLE grades (
    university VARCHAR(30),
    first_name VARCHAR(30),
    last_name VARCHAR(30),
    subject VARCHAR(30),
    grade VARCHAR (10));
    """ 
)

In [11]:
engine.execute(
    """
    INSERT INTO grades(first_name, last_name, subject, grade, university)
    SELECT rg.first_name, rg.last_name, sb.canonical_subject, rg.grade, rg.university
    FROM raw_grades rg, subjects sb
    WHERE LOWER(rg.subject) = sb.subject;
    """)

In [12]:
grades_df = pd.read_sql_query('SELECT * FROM grades', engine)
grades_df.sample(n=12)

,university,first_name,last_name,subject,grade
16257,uni2,wgznsmsmys,fvgmnwwkeh,physics,91.0
4165,uni3,flnlgibx,rerhcngcp,bus. administration,64.0
9355,uni3,jrkw,avpi,mathematics,84.0
10881,uni1,nchl,kdvmrwwpke,mathematics,B
20330,uni3,glcqe,otdqi,soc studies,84.0
3298,uni2,esurqnzmvj,peuzdylw,biology,8.0
19762,uni3,luupkbomvt,vppwvjb,soc studies,82.0
18671,uni2,f,ckbedmxth,soc studies,24.0
5157,uni3,td,lcyue,computer science,8.0
16368,uni3,hahwsnn,bdaa,physics,14.0


In [13]:
grades_df.shape

(20480, 5)

## Top 3 Popular Classes Spanning All Universities

In [14]:
pd.read_sql_query(
    """
    SELECT subject, count(*) as enrolments
    FROM grades
    GROUP BY subject
    ORDER BY enrolments DESC
    LIMIT 3;
    """
    , engine)

,subject,enrolments
0,mathematics,3072
1,physics,3072
2,soc studies,3072


## Courses Offered by No More than One University

In [15]:
pd.read_sql_query(
    """
    SELECT subject, university_count
    FROM (SELECT subject, count(*) as university_count
          FROM (SELECT DISTINCT university, subject
                FROM grades) university_subject
          GROUP BY subject) subject_university_count
    WHERE university_count = 1
    ORDER BY subject;
    """
    , engine)

,subject,university_count
0,ancient history,1
1,biology,1
2,bus. administration,1
3,computer science,1
4,economics,1
5,history,1
6,neural networks,1


## Passing Rate at Each University

### Passing Count per University

In [16]:
university_passing_count_df = pd.read_sql_query(
    """
    SELECT university, count(*) passing_count
    FROM (SELECT * FROM grades
          WHERE grade IN ('A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-') OR 
                (grade ~ '[0-9]+\.[0-9]+' AND grade::numeric >= 50)) university_passing_count
    GROUP BY university;
    """
    , engine)
university_passing_count_df

,university,passing_count
0,uni3,2684
1,uni2,3633
2,uni1,3672


### Student Count per University

In [17]:
university_count_df = pd.read_sql_query(
    """
    SELECT university, count(*) student_count
    FROM grades
    GROUP BY university;
    """
    , engine)
university_count_df

,university,student_count
0,uni3,6144
1,uni2,8192
2,uni1,6144


We need to join these results to be able to compute the passing rate:

In [18]:
university_stats_df = \
    university_count_df.set_index('university')\
        .join(university_passing_count_df.set_index('university'))
university_stats_df

,student_count,passing_count
university,,
uni3,6144,2684
uni2,8192,3633
uni1,6144,3672


### Final Results

Here are the passing rates per university:

In [19]:
university_stats_df['passing_rate'] = \
    university_stats_df\
        .apply(lambda row: row.passing_count / row.student_count * 100, axis=1)\
        .round(2)
university_stats_df

,student_count,passing_count,passing_rate
university,,,
uni3,6144,2684,43.68
uni2,8192,3633,44.35
uni1,6144,3672,59.77
